### Notebook for homology searches for DNA meythlation machinery

This needs to have Java 11 in the path and for example run in the pycoMeth environment only

In [1]:
import os
from Bio import SeqIO
import pandas as pd
import numpy as np
import re

In [2]:
notebook_path = os.path.abspath(".")

In [3]:
IN_DIR = os.path.abspath('../../analyses/methylation_machinery/')
OUT_DIR = os.path.abspath('../../analyses/methylation_machinery/')
GENOME_DIR = os.path.abspath('../../data/genomic_resources/')

In [17]:
Pgt_protein_fn = os.path.abspath('../../data/genomic_resources/Puccinia_graminis_tritici_21-0.proteins.fa')
SizmA_seeds_fn = os.path.abspath('../../analyses/methylation_machinery/6mA_methylation_demethyltion_query.fasta')
FivemC_seeds_fn = SizmA_seeds_fn

In [18]:
n_threads = 20
blast_outfmt6_headers = "qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore".split(' ')

In [19]:
###write a function that takes the interpro TSV and returns a dict of domains for a specific search engine
def interpro_accession_dict(fn):
    header = ['P-ID', 'md5', 'len', 'analysis', 'accession', 'description', 'start', 'stop', 'score', 'status' , 'date',
         'Interpro_accession', 'Interpro_description']
    df = pd.read_csv(fn, sep='\t', header =None, names= header).dropna()
    return dict(zip(df.groupby('P-ID')['Interpro_accession'].unique().index, df.groupby('P-ID')['Interpro_accession'].unique()))

In [20]:
###write a function that takes the interpro TSV and returns a dict of domains for a specific search engine
def interpro_analysis_dict(fn, analysis):
    header = ['P-ID', 'md5', 'len', 'analysis', 'accession', 'description', 'start', 'stop', 'score', 'status' , 'date',
         'Interpro_accession', 'Interpro_description']
    df = pd.read_csv(fn, sep='\t', header =None, names= header).dropna()
    grouped = df[df.analysis == analysis].groupby('P-ID')
    return dict(zip(grouped['analysis'].unique().index, grouped['accession'].unique()))

### Here the blast analysis starts

In [21]:
os.chdir(OUT_DIR)

In [22]:
!makeblastdb -dbtype prot -in {Pgt_protein_fn}



Building a new DB, current time: 04/29/2020 17:55:26
New DB name:   /home/jamila/jamila_Storage/data/genomic_resources/Puccinia_graminis_tritici_21-0.proteins.fa
New DB title:  /home/jamila/jamila_Storage/data/genomic_resources/Puccinia_graminis_tritici_21-0.proteins.fa
Sequence type: Protein
Deleted existing Protein BLAST database named /home/jamila/jamila_Storage/data/genomic_resources/Puccinia_graminis_tritici_21-0.proteins.fa
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 37832 sequences in 0.6715 seconds.


In [25]:
#define file names
SixmA_outfmt_6_fn = 'Puccinia_graminis_tritici_21-0.proteins.6mA_methylation_demethyltion_query.blastp.outfmt6'
SixmA_outfmt_6_fn = os.path.join(OUT_DIR, SixmA_outfmt_6_fn)
FivemC_outfmt_6_fn = SixmA_outfmt_6_fn

In [24]:
#run blast
!blastp -num_threads 20 -outfmt 6 -query {SizmA_seeds_fn} -db {Pgt_protein_fn} > {SixmA_outfmt_6_fn}

In [26]:
!head {SixmA_outfmt_6_fn}

sp|Q09956|DAMT1_CAEEL	PGT21_005363-T1	30.769	208	113	9	160	339	253	457	4.65e-14	73.2
sp|Q09956|DAMT1_CAEEL	PGT21_004684-T1	30.288	208	114	9	160	339	229	433	5.31e-13	69.7
sp|Q09956|DAMT1_CAEEL	PGT21_023926-T1	32.727	55	34	2	161	212	8	62	1.7	30.0
sp|Q09956|DAMT1_CAEEL	PGT21_024228-T1	30.435	46	32	0	209	254	469	514	3.8	29.6
sp|Q09956|DAMT1_CAEEL	PGT21_033706-T1	30.909	55	35	2	161	212	249	303	8.0	28.5
sp|Q09956|DAMT1_CAEEL	PGT21_003002-T1	32.857	70	41	2	257	325	35	99	8.5	28.5
sp|Q09956|DAMT1_CAEEL	PGT21_027835-T2	37.500	32	17	1	322	350	647	678	8.7	28.5
sp|Q09956|DAMT1_CAEEL	PGT21_014712-T1	29.592	98	52	4	234	323	275	363	9.4	28.1
sp|Q9Y5N5|N6MT1_HUMAN	PGT21_005253-T1	44.186	215	109	6	8	213	4	216	1.21e-50	163
sp|Q9Y5N5|N6MT1_HUMAN	PGT21_005713-T1	44.186	215	109	6	8	213	4	216	1.27e-50	163


### Downstream filtering of blast resutls

In [27]:
FivemC_blast_df = pd.read_csv(FivemC_outfmt_6_fn, header = None, names=blast_outfmt6_headers, sep='\t' )

In [28]:
FivemC_blast_df.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,sp|Q09956|DAMT1_CAEEL,PGT21_005363-T1,30.769,208,113,9,160,339,253,457,4.650000e-14,73.2
1,sp|Q09956|DAMT1_CAEEL,PGT21_004684-T1,30.288,208,114,9,160,339,229,433,5.310000e-13,69.7
2,sp|Q09956|DAMT1_CAEEL,PGT21_023926-T1,32.727,55,34,2,161,212,8,62,1.700000e+00,30.0
3,sp|Q09956|DAMT1_CAEEL,PGT21_024228-T1,30.435,46,32,0,209,254,469,514,3.800000e+00,29.6
4,sp|Q09956|DAMT1_CAEEL,PGT21_033706-T1,30.909,55,35,2,161,212,249,303,8.000000e+00,28.5


In [29]:
#filtering of blast_df
FivemC_stringent_blast_df =  FivemC_blast_df[FivemC_blast_df.evalue < 1e-10].copy() 

In [30]:
FivemC_stringent_blast_df.groupby('qseqid')['sseqid'].count()

qseqid
sp|Q09956|DAMT1_CAEEL    2
sp|Q13686|ALKB1_HUMAN    2
sp|Q9Y5N5|N6MT1_HUMAN    2
Name: sseqid, dtype: int64

In [31]:
FivemC_stringent_blast_df.sseqid.unique()

array(['PGT21_005363-T1', 'PGT21_004684-T1', 'PGT21_005253-T1',
       'PGT21_005713-T1', 'PGT21_022089-T1', 'PGT21_022364-T1'],
      dtype=object)

In [32]:
FivemC_seeds_ids = []
for seq in SeqIO.parse(FivemC_seeds_fn, 'fasta'):
    FivemC_seeds_ids.append(seq.id)

In [33]:
not_present = set(FivemC_seeds_ids) - set(FivemC_stringent_blast_df.qseqid.unique())

In [34]:
not_present

{'sp|Q8MNT9|NMAD1_CAEEL'}

In [35]:
set(FivemC_seeds_ids) - set(FivemC_blast_df[FivemC_blast_df.evalue < 1e-2].qseqid.unique())

{'sp|Q8MNT9|NMAD1_CAEEL'}

In [39]:
##pull out fasta sequence of all the hits
e_value = 0.01
FivemC_Pgt_protein_hit_fn = 'Puccinia_graminis_tritici_21-0.proteins.6mA_methylation_demethyltion_query.blastp-%s.fasta' % e_value
FivemC_Pgt_protein_hit_fn = os.path.join(OUT_DIR, FivemC_Pgt_protein_hit_fn)

In [40]:
blast_df = FivemC_blast_df

In [41]:
###get all the hits once and subset the blast with the e-value selected
hit_ids = blast_df[blast_df.evalue < e_value].sseqid.unique()
hit_list = []
sub_blast_df = blast_df[blast_df.evalue < e_value].copy()
for seq in SeqIO.parse(Pgt_protein_fn, 'fasta'):
    if seq.id in hit_ids:
        print(seq.id)
        hit_list.append(seq)
SeqIO.write(hit_list, FivemC_Pgt_protein_hit_fn, 'fasta')

PGT21_004684-T1
PGT21_005253-T1
PGT21_005363-T1
PGT21_005713-T1
PGT21_022089-T1
PGT21_022364-T1


6

In [42]:
sub_blast_df

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,sp|Q09956|DAMT1_CAEEL,PGT21_005363-T1,30.769,208,113,9,160,339,253,457,4.650000e-14,73.2
1,sp|Q09956|DAMT1_CAEEL,PGT21_004684-T1,30.288,208,114,9,160,339,229,433,5.310000e-13,69.7
8,sp|Q9Y5N5|N6MT1_HUMAN,PGT21_005253-T1,44.186,215,109,6,8,213,4,216,1.210000e-50,163.0
9,sp|Q9Y5N5|N6MT1_HUMAN,PGT21_005713-T1,44.186,215,109,6,8,213,4,216,1.270000e-50,163.0
34,sp|Q13686|ALKB1_HUMAN,PGT21_022089-T1,29.439,214,99,7,167,347,308,502,9.800000e-21,93.6
36,sp|Q13686|ALKB1_HUMAN,PGT21_022364-T1,29.439,214,98,7,167,347,327,520,7.800000e-20,90.9


### Pull in haplotype information

In [43]:
pgt_gff3_fn = os.path.join('../../data/genomic_resources/Puccinia_graminis_tritici_21-0.gff3')

In [44]:
with open(pgt_gff3_fn, 'r') as fh:
    haplotype_dict = {}
    for line in fh:
        line = line.rstrip()
        if any(s in line for s in hit_ids):
            for hit in hit_ids:
                if hit in line:
                    haplotype_dict[hit] = line.split('\t')[0][-1]

In [45]:
len(haplotype_dict.values()) == len(hit_ids)

True

In [46]:
sub_blast_df['shaplotype'] = sub_blast_df.sseqid.map(haplotype_dict)

In [47]:
#get the locus id for loci with multiple transcripts
sub_blast_df['sseqid_locus'] = [x.split('-')[0] for x in sub_blast_df.sseqid]

In [48]:
#only keep the transcript witht the best hit
sub_blast_df.drop_duplicates(['qseqid', 'sseqid_locus'], keep='first', inplace = True)

### Do Interpro scan on command line

In [49]:
interpro5 = '/home/jamila/anaconda3/downloads/interproscan-5.42-78.0/interproscan.sh'

In [50]:
TMP_DIR = os.path.join(OUT_DIR, 'tmp')
if not os.path.exists(TMP_DIR):
    os.mkdir(TMP_DIR)

In [51]:
Pgt_protein_hit_intrpro_fn = os.path.join(TMP_DIR, os.path.basename(FivemC_Pgt_protein_hit_fn).replace('.fasta', '.interpro5.tsv'))
FivemC_seeds_intrpro_fn = os.path.join(TMP_DIR, os.path.basename(FivemC_seeds_fn).replace('.fasta', '.interpro5.tsv'))

Run interpro on both set of protein files

In [52]:
!head {FivemC_Pgt_protein_hit_fn}

>PGT21_004684-T1 PGT21_004684
MSQRSEPSSVLLEILLPEKNLTAYLLDPFVPLLRGFDETTRVSVSRPPSEPHIICTPTTS
ASTTTNRENRPTKRQRSNNEPPSASAHSCQPGESAETSQIHPLNGFQLEAVHHASIKGDI
QQAVDAIRERWIDECSRDGHSEWWSDPHLLRHSAVAQLEWLPSPEIKRSEVWVDWAALTR
DTFQVRCKTQSDLNQSSIRVSESGITCTNSSSTSISQTFLPEKKCSITLPKRSGFSLATM
ENFKSEVFGLNHSPGWDAVIIDPPWQNKSATRGGKYRTVELYDLFKLDLPGILGENGGKR
ALIAVWVTNRPKYRRFLKNKFFPDCHVQGPYSEWYWVKITASPTVKDDQPALSEGGKPLF
DLNSTSPRRCYEGLVLGWYIPPSLRPEVRLSELPPKIFLSVPLGHSRKPNIIDLLAPHLP
SDPSILELFSRSVSGLTSLAKPLEDKSAVLSPMKGIWHSVGDESPKFNVAPWVVLDSSIN
TPSLSDP


In [53]:
!bash {interpro5} -cpu 4 -i {FivemC_Pgt_protein_hit_fn} -f tsv -iprlookup -o {Pgt_protein_hit_intrpro_fn}

29/04/2020 17:57:27:504 Welcome to InterProScan-5.42-78.0
29/04/2020 17:57:27:506 Running InterProScan v5 in STANDALONE mode... on Linux
29/04/2020 17:57:41:140 Loading file /home/jamila/jamila_Storage/analyses/methylation_machinery/Puccinia_graminis_tritici_21-0.proteins.6mA_methylation_demethyltion_query.blastp-0.01.fasta
29/04/2020 17:57:41:143 Running the following analyses:
[CDD-3.17,Coils-2.2.1,Gene3D-4.2.0,Hamap-2020_01,MobiDBLite-2.0,PANTHER-14.1,Pfam-32.0,PIRSF-3.02,PRINTS-42.0,ProSitePatterns-2019_11,ProSiteProfiles-2019_11,SFLD-4,SMART-7.1,SUPERFAMILY-1.75,TIGRFAM-15.0]
Available matches will be retrieved from the pre-calculated match lookup service.

Matches for any sequences that are not represented in the lookup service will be calculated locally.
29/04/2020 17:58:07:015 25% completed
29/04/2020 17:58:45:819 51% completed
29/04/2020 17:59:10:012 75% completed
29/04/2020 17:59:20:157 92% completed
29/04/2020 17:59:30:121 100% done:  InterProScan analyses completed 



In [54]:
!bash {interpro5} -cpu 4 -i {FivemC_seeds_fn} -f tsv -iprlookup -o {FivemC_seeds_intrpro_fn}

29/04/2020 17:59:31:771 Welcome to InterProScan-5.42-78.0
29/04/2020 17:59:31:773 Running InterProScan v5 in STANDALONE mode... on Linux
29/04/2020 17:59:47:699 Loading file /home/jamila/jamila_Storage/analyses/methylation_machinery/6mA_methylation_demethyltion_query.fasta
29/04/2020 17:59:47:703 Running the following analyses:
[CDD-3.17,Coils-2.2.1,Gene3D-4.2.0,Hamap-2020_01,MobiDBLite-2.0,PANTHER-14.1,Pfam-32.0,PIRSF-3.02,PRINTS-42.0,ProSitePatterns-2019_11,ProSiteProfiles-2019_11,SFLD-4,SMART-7.1,SUPERFAMILY-1.75,TIGRFAM-15.0]
Available matches will be retrieved from the pre-calculated match lookup service.

Matches for any sequences that are not represented in the lookup service will be calculated locally.
29/04/2020 18:00:07:080 25% completed
29/04/2020 18:00:40:309 50% completed
29/04/2020 18:00:56:403 75% completed
29/04/2020 18:01:17:485 92% completed
29/04/2020 18:01:26:813 100% done:  InterProScan analyses completed 



In [55]:
#pull in interpro results and add them to the dataframe
sub_blast_df['q_pfam'] = sub_blast_df.qseqid.map(interpro_analysis_dict(FivemC_seeds_intrpro_fn, 'Pfam'))
sub_blast_df['q_interpro'] = sub_blast_df.qseqid.map(interpro_accession_dict(FivemC_seeds_intrpro_fn))
sub_blast_df['s_pfam'] = sub_blast_df.sseqid.map(interpro_analysis_dict(Pgt_protein_hit_intrpro_fn, 'Pfam'))
sub_blast_df['s_interpro'] = sub_blast_df.sseqid.map(interpro_accession_dict(Pgt_protein_hit_intrpro_fn))


In [56]:
#do some cosmetics on the the dataframe for proteins without interpro /pfam domains because pandas is wierd sometimes.
for cln in ['q_pfam', 'q_interpro', 's_pfam','s_interpro']:
    if sub_blast_df[cln].isna().sum():
        sub_blast_df.loc[sub_blast_df[sub_blast_df[cln].isna()].index, cln] = [ [[]] * sub_blast_df[cln].isna().sum() ]

In [57]:
#calculate the fraction of overlapping interpro/pfam domains between query sequences and hits
sub_blast_df['pfam_int'] = sub_blast_df.apply(lambda row: set(row['q_pfam']).intersection(set(row['s_pfam'])) , axis=1)
sub_blast_df['pfam_int_frac'] = sub_blast_df['pfam_int'].apply(lambda x: len(x)) / sub_blast_df['q_pfam'].apply(lambda x: len(x))
sub_blast_df['interpro_int'] = sub_blast_df.apply(lambda row: set(row['q_interpro']).intersection(set(row['s_interpro'])) , axis=1)
sub_blast_df['interpro_int_frac'] = sub_blast_df['interpro_int'].apply(lambda x: len(x)) / sub_blast_df['q_interpro'].apply(lambda x: len(x))

In [58]:
sub_blast_df.iloc[:,[0,1,10, 17, 18,19]].head(30)

,qseqid,sseqid,evalue,s_interpro,pfam_int,pfam_int_frac
0,sp|Q09956|DAMT1_CAEEL,PGT21_005363-T1,4.650000e-14,[IPR007757],{PF05063},1.0
1,sp|Q09956|DAMT1_CAEEL,PGT21_004684-T1,5.310000e-13,[IPR007757],{PF05063},1.0
8,sp|Q9Y5N5|N6MT1_HUMAN,PGT21_005253-T1,1.210000e-50,"[IPR004557, IPR007848]",{PF05175},1.0
9,sp|Q9Y5N5|N6MT1_HUMAN,PGT21_005713-T1,1.270000e-50,"[IPR002052, IPR004557, IPR007848]",{PF05175},1.0
34,sp|Q13686|ALKB1_HUMAN,PGT21_022089-T1,9.800000e-21,"[IPR027450, IPR005123]",{PF13532},1.0
36,sp|Q13686|ALKB1_HUMAN,PGT21_022364-T1,7.800000e-20,"[IPR027450, IPR005123]",{PF13532},1.0


In [59]:
#filter the dataframe to have only hits that have the best possible interpro domains fractions
pfam_filt_df = sub_blast_df[sub_blast_df.groupby('qseqid')['interpro_int_frac'].transform(max) == sub_blast_df['interpro_int_frac']]

In [60]:
##look at how many hits per query sequence are still left
pfam_filt_df.groupby('qseqid')['sseqid'].count()

qseqid
sp|Q09956|DAMT1_CAEEL    2
sp|Q13686|ALKB1_HUMAN    2
sp|Q9Y5N5|N6MT1_HUMAN    1
Name: sseqid, dtype: int64

In [61]:
best_sseq_df = pfam_filt_df[pfam_filt_df.groupby('sseqid')['interpro_int_frac'].transform(max) == pfam_filt_df['interpro_int_frac']]

In [62]:
pgt_match_list = []
DNA_seed_list = []
haplotype_list = []
match_type_list = []
for seed_gene, pgt_gene  in zip(best_sseq_df.qseqid, best_sseq_df.sseqid): 
    if not pgt_gene.endswith('-T2'):
        DNA_seed_list.append(seed_gene)
        pgt_match_list.append(pgt_gene)
        match_type_list.append('blast')

In [63]:
pgt_match_series = pd.Series(pgt_match_list, name="Pgt_match")
DNA_seed_series = pd.Series(DNA_seed_list, name='Seed_ID')
haplotype_series = pd.Series(haplotype_list, name='haplotype')
match_type_series = pd.Series(match_type_list, name='Match_type')

/home/jamila/anaconda3/envs/pycoMeth/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
out_df = pd.concat([DNA_seed_series, pgt_match_series, haplotype_series, match_type_series], axis =1)

In [65]:
out_fn = os.path.join(OUT_DIR, '%s_orthologs.Pgt21-0.tsv' %os.path.basename(FivemC_seeds_fn).replace('.fasta', '') )

In [66]:
out_df.to_csv(out_fn, sep='\t', index=None)

In [67]:
!head {out_fn}

Seed_ID	Pgt_match	haplotype	Match_type
sp|Q09956|DAMT1_CAEEL	PGT21_005363-T1		blast
sp|Q09956|DAMT1_CAEEL	PGT21_004684-T1		blast
sp|Q9Y5N5|N6MT1_HUMAN	PGT21_005713-T1		blast
sp|Q13686|ALKB1_HUMAN	PGT21_022089-T1		blast
sp|Q13686|ALKB1_HUMAN	PGT21_022364-T1		blast
